# Mobile inverted pendulum python dynamics
author:SeongHyeon Kim

program:PYDY balancing robot simulator


필요한 라이브러리 포함


In [1]:
from sympy import sin, cos, symbols, solve, Matrix, init_printing
from sympy.physics.mechanics import *
import numpy as np
from sympy.physics.vector import ReferenceFrame

In [2]:
init_vprinting()

# Reference frames

N은 외부의 기준 프레임. 속도를 0으로

In [3]:
#Newtonian reference frame
N=ReferenceFrame('N')

# Define a world coordinate origin
No = Point('No')
No.set_vel(N, 0)

# Variables

변수선언

In [4]:
x = dynamicsymbols('x')            # F1> measure number of Velocity of P in N
xd = dynamicsymbols('xd')
xd2 = dynamicsymbols('xdd')
theta = dynamicsymbols('theta')    # F2> measure number of Angular velocity of P in N
thetad = dynamicsymbols('thetad')
thetad2 = dynamicsymbols('thetadd')
delta = dynamicsymbols('delta')    # N3> measure number of Angular velocity of P in N
deltad = dynamicsymbols('deltad')
deltad2 = dynamicsymbols('deltadd')
wL = dynamicsymbols('wL')          # Angular velocity of L in N
wLd = dynamicsymbols('wL', 1)
wLd2 = dynamicsymbols('wL', 2)
wR = dynamicsymbols('wR')          # Angular velocity of R in N
wRd = dynamicsymbols('wR', 1)
wRd2 = dynamicsymbols('wR', 2)
TL, TR = dynamicsymbols('tL tR')   #Torques on L, R
g = symbols('g', real=True)        #Gravitational acceleration
d = symbols('d', real=True)        # Distance from point I to Lo (or Ro)

I2P = symbols('I2P', real=True)        # Distance from point I to mass center of P
t = symbols('t')
I = Point('i')

In [5]:

# Create q and dq vectors
q = Matrix([x, delta, theta, wL, wR])
dq = q.diff()

In [6]:
K,J = symbols('K J')

In [7]:
#Robot heading Frame
F= N.orientnew('H', 'Axis',[delta, N.z] )

# Robot Reference Frame(Center of mass)
P = F.orientnew('P', 'Axis', [theta, F.y])

F.set_ang_vel(N, deltad*N.z)        # Angular velocity of F in N
P.set_ang_vel(F, thetad*F.y)        # Angular velocity of P in F


V_I_N = I.set_vel(N, xd*F.x)             # Velocity of point I in N



Lo = I.locatenew('Lo', d*F.y)                    # Point I to center of wheel L
Ro = I.locatenew('Ro', -d*F.y)                   # Point I to center of wheel R


V_I_N

기준프레임.dcm(상대프레임)

In [8]:
N.dcm(F)

⎡cos(δ)  -sin(δ)  0⎤
⎢                  ⎥
⎢sin(δ)  cos(δ)   0⎥
⎢                  ⎥
⎣  0        0     1⎦

In [9]:
F.dcm(P)

⎡cos(θ)   0  sin(θ)⎤
⎢                  ⎥
⎢   0     1    0   ⎥
⎢                  ⎥
⎣-sin(θ)  0  cos(θ)⎦

여기까진 확인

## Wheel variables

바퀴지름,질량, 관성모멘트 값

관성모멘트함수=inertia(기준 좌표, xx, yy,zz회전)

In [10]:
R =  ReferenceFrame('R')                           #wheel frames
L =  ReferenceFrame('L')

In [11]:
#Constans for the wheels
wR = symbols('wR', real=True)                      #Radius of wheels  
M = symbols('M')                                 #wheel mass
I_L = inertia(F, K,J,K)                          #Wheel inertia 
I_R = inertia(F, K,J,K)


## Body variables

바퀴간 거리, 질량, 관성모멘트

In [12]:
#Constants for the Robot body
I = No.locatenew('I', x*F.x)                      # Point No to I
Po = I.locatenew('Po', I2P*P.z)                     # Point I to mass center of P
m_b = symbols('m_b')                              #Mass of the body
I1, I2, I3 = symbols('I1 I2 I3')                  #Moments of inertia of body

P는 로봇의 무게중심기준계.F는 로봇 방향 기준계. Ff는 앞바퀴의 기준계.

Ff의 속도 설정 diff() 는 미분을 의미한다.

# Modeling Wheel (Joints)

## Wheel Frames & Angular Velocities

바퀴위치, 바퀴회전, 바퀴의 프레임 선언, 바퀴 강체선언


기준.locatenew('이름', 이동*단위백터)


In [13]:

# Create reference frames for wheels
L.set_ang_vel(F, wLd*F.y)    # wL is related to w and v due to rolling

R.set_ang_vel(F, wRd*F.y)    # wR is related to w and v due to rolling



In [14]:
 #바닥과 닿는점
LN = Lo.locatenew('LN', -wR*F.z)   # Center of wheel L to contact point LN

RN = Ro.locatenew('RN', -wR*F.z)   # Center of wheel R to contact point RN

RigidBody =강체 선언 (입자로 저장된다고 한다)


 B = RigidBody ( '이름' ,질량 중심, 프레임 ,질량, ( I , P ))
 

In [15]:
# Create rigid bodies for wheels
Wheel_L = RigidBody('Wheel_L', Lo, L, M, (I_L, Lo))
Wheel_R = RigidBody('Wheel_R', Ro, R, M, (I_R, Ro))

## Modeling body

In [16]:
# Calculate inertia of body
Ib = inertia(F, I1, I2, I3)

# Center of mass of body   =Po

# Create a rigid body object for body
Body = RigidBody('Body', Po, P, m_b, (Ib, Po))

## Nonholonomic Constranits

In [17]:
V_LN_N = LN.v2pt_theory(Lo,N,L)          # Velocity of contact point of L

V_RN_N = RN.v2pt_theory(Ro,N,R)          # Velocity of contact point of R

In [21]:
# Express the velocity of points in the inertial frame
con1 = LN.vel(N).express(N).args[0][0]
con2 = RN.vel(N).express(N).args[0][0]
# Create a matrix of constraints
constraints = con1.col_join(con2)
constraints

⎡(-d⋅deltad - wR⋅wL̇ + xd)⋅cos(δ)⎤
⎢                               ⎥
⎢(-d⋅deltad - wR⋅wL̇ + xd)⋅sin(δ)⎥
⎢                               ⎥
⎢               0               ⎥
⎢                               ⎥
⎢(d⋅deltad - wR⋅wṘ + xd)⋅cos(δ) ⎥
⎢                               ⎥
⎢(d⋅deltad - wR⋅wṘ + xd)⋅sin(δ) ⎥
⎢                               ⎥
⎣               0               ⎦

In [22]:
# Solve for dx, dy, and dtheta in terms of dphi1 and dphi2
sol = solve(constraints, dq[:3])

# Split the resulting dict into a rhs and lhs, that are equivalent
sol_rhs = Matrix(list(sol.values()))
sol_lhs = Matrix(list(sol.keys()))

# Since sol_rhs = sol_lhs --> sol_rhs - sol_lhs = 0
# This forms the basis of our constraint matrix.
# Combining, and solving for a linear representation:
c = (sol_rhs - sol_lhs).jacobian(dq[:5])
c

AttributeError: 'list' object has no attribute 'values'

In [ ]:
# Constraint Equations
coneqs = (c*dq)
coneqs

# Kinematical Differential Equations

In [ ]:
kinematical_differential_equations = [xd - x.diff(),
                                      thetad - theta.diff(),
                                      deltad - delta.diff()]
kinematical_differential_equations

### 각각의 Mass center 에서의 속도


v2pt_theory는 선속도를 의미


$$^I\mathbf{v}^{P_2} = ^I\mathbf{v}^{P_1} + ^I\omega^A \times \mathbf{r}^{\frac{P_2}{P_1}}$$

The `Point.v2pt_theory()` method makes it easy to do this calculation.

측정점.v2pt_theory(회전 중심점, 관찰틀, 회전틀(돌아가는 애))



속도를 측정하는게 아니고 부여하는거 같다 좀더 알아보자


In [ ]:

V_I_N = I.set_vel(N, xd*F.x)             # Velocity of point I in N



L1 = I.locatenew('L1', d*F.y)                    # Point I to center of wheel L
R1 = I.locatenew('R1', -d*F.y)                   # Point I to center of wheel R

In [ ]:

V_PO_N = Po.v2pt_theory(I,N,P)           # Velocity of center of mass of P

V_LO_N = Lo.v2pt_theory(I, N,P,)         # Velocity of center of L

V_RO_N = Ro.v2pt_theory(I,N,P)           # Velocity of center of R


### 토크와 힘 작용

In [ ]:
#중력벡터
Gravity = -g * N.z
fL = []

fL.extend([(P, -g*m_b*N.z),(R, -g*M*N.z),(L, -g*M*N.z)],)

In [ ]:

fL.append( (L, TL*F.y))     #Torques on L

    
fL.append((R, TR*F.y))    #Torques on R

fL

# 운동방정식으로 만들자

## Generalized Forces

[토크힘] x [속도,각속도]

[Cm속도, 앞바퀴각속도, 뒷바퀴각속도, 몸체 각속도] x [중력, 앞바퀴토크, 뒷바퀴토크, Cm토크] 

VnWxTnF

In [ ]:
from __future__ import print_function, division

In [ ]:
from sympy import trigsimp
from sympy.physics.mechanics import KanesMethod

In [ ]:
from sympy.physics.vector import init_vprinting

In [ ]:
q = [x, theta, delta]
q

In [ ]:
qd = [xd, thetad, deltad]
qd

In [ ]:
kane = KanesMethod(N, q, qd, kinematical_differential_equations)

In [ ]:
fL

In [ ]:
bodies = [Body, Wheel_L, Wheel_R]
bodies

In [ ]:
fr, frstar = kane.kanes_equations(bodies, fL)

### F + F* =0 식 (Kanes equation)

In [ ]:
trigsimp(fr + frstar)

In [ ]:
mass_matrix = trigsimp(kane.mass_matrix_full)
mass_matrix

In [ ]:
forcing_vector = trigsimp(kane.forcing_full)
forcing_vector

In [ ]:
find_dynamicsymbols(kane.forcing)

In [ ]:
fr

In [ ]:
frstar

# 선형화

In [ ]:
from numpy import array, zeros, eye, asarray, dot, rad2deg
from numpy.linalg import inv

In [ ]:
%matplotlib inline

In [ ]:
from matplotlib.pyplot import plot, xlabel, ylabel, legend, rcParams

In [ ]:
rcParams['figure.figsize'] = (14, 8)

In [ ]:
from sympy import simplify, Matrix, matrix2numpy
from sympy.physics.vector import init_vprinting, vlatex

In [ ]:
%precision 3

In [ ]:
KanesMethod?

In [ ]:
equilibrium_point = zeros(len(q + dq))
equilibrium_point

In [ ]:
equilibrium_dict = dict(zip(q + dq, equilibrium_point))
equilibrium_dict

In [ ]:
M, F_A, F_B, r = kane.linearize(new_method=True, op_point=equilibrium_dict)

In [ ]:
simplify(M)

In [ ]:
simplify(F_A)

In [ ]:
simplify(F_B) 

# Simulation

In [ ]:
from sympy import Dummy, lambdify
from numpy import array, hstack, zeros, linspace, pi
from numpy.linalg import solve
from scipy.integrate import odeint

### System parameter

In [ ]:
parameters= [g, m_b ,m_wf, m_wr]
parameter_vals=[9.81, 10., 5., 3.]
parameter_dict = dict(zip(parameters, parameter_vals))
parameter_dict

# Control

In [ ]:
import control
from numpy import dot
from numpy.linalg import matrix_rank